# Ocular Disease Intelligent Recognition (ODIR)
## Using Convolutional Neural Network to Predict Ocular Diseases

Project by: 
    22BAI1213 Joshua S Raju, 22BRS1372 Md Rameez Haider, 22BAI1427 Shubham Yadav, 22BAI1425 Anton K Sam

This particular CNN model aims to predict three types of Ocular Diseases:
1. Cataract

Dataset : https://www.kaggle.com/datasets/andrewmvd/ocular-disease-recognition-odir5k

## Importing Dependencies

In [1]:
import numpy as np
import os

import imghdr

import cv2
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Reading Image Data

In [3]:
data_dir = 'imgdata_v2'
os.listdir(data_dir)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'im'

img_exts = ['jpeg',  'jpg', 'png']
img_exts

In [ ]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        img_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(img_path)
            ext = imghdr.what(img_path)
            if ext not in img_exts:
                print('Image not in appropriate fromat {}'.format(img_path))
                os.remove(img_path)
        except Exception as e:
            print('Issue with the image {}'.format(img_path))

In [ ]:
classes = ['Cataract', 'Normal']
classes

In [ ]:
## Classifying Image Data into a Classified Image Dataset

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
batch = data.as_numpy_iterator().next()
len(batch)

## Displaying Images from Dataset

In [ ]:
fig, axs = plt.subplots(5,5, figsize=(15,15))
for idx, img in enumerate(batch[0][:25]):
    row = idx // 5
    col = idx % 5
    axs[row, col].imshow(img.astype(int))
    axs[row, col].axis('off')
    axs[row, col].set_title(classes[batch[1][idx]], fontsize = 10)
    fig.suptitle('BATCH IMAGES', fontsize=20)

## Image Normalization

In [ ]:
data.as_numpy_iterator().next()[0].max(), data.as_numpy_iterator().next()[0].min()

In [ ]:
data = data.map(lambda x, y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()[0]

In [ ]:
data.as_numpy_iterator().next()[0].max(), data.as_numpy_iterator().next()[0].min()

In [ ]:
len(data)

## Splitting Dataset 

In [ ]:
training_size = int(len(data)*0.7)
val_size = int(len(data)*0.2) + 1
test_size = int(len(data)*0.1)

training_size + val_size + test_size

In [ ]:
training_data = data.take(training_size)
val_data = data.skip(training_size).take(val_size)
test_data =  data.skip(training_size + val_size).take(test_size)

## Creating CNN Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
data.as_numpy_iterator().next()[0][0].shape

In [ ]:
model.add(Conv2D(32, (3,3), 1, activation = 'relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())
model.add(Dropout(0.5))


model.add(Flatten())

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

## Training The Model

In [ ]:
log_dir = 'LogDir'

In [ ]:
tensorflowboard_callback = tf.keras.callbacks.TensorBoard(log_dir)

In [ ]:
hist = model.fit(training_data, epochs = 30, validation_data = val_data, callbacks=[tensorflowboard_callback] )

In [ ]:
hist.history

## Model Performance 

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10,10))
axs[0].plot(model_fit.history['loss'], color='red', label='Loss')
axs[0].plot(model_fit.history['val_loss'], color='orange', label='Validation Loss')
axs[0].legend(loc='upper right')

axs[1].plot(model_fit.history['accuracy'], 
             color='green', 
             label='Accuracy')
axs[1].plot(model_fit.history['val_accuracy'], 
             color='teal', 
             label='Validation Accuracy')
axs[1].legend(loc='upper left')

## Evaluating Model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, SpecificityAtSensivity

pre = Precision()
acc = BinaryAccuracy()
rec = Recall()
spe = SpecificityAtSensitivity(0.5)

In [ ]:
for batch in test_data.as_numpy_iterator():
    X, y = batch
    y_pred = model.predict(X)
    pre.update_state(y, y_pred)
    rec.update_state(y, y_pred)
    acc.update_state(y, y_pred)
    spe.update_state(y, y_pred)

In [ ]:
precision = pre.result().numpy()
recall = rec.result().numpy()
f1 = 2 * (precision*recall) / (precision+recall)

print('Accuracy\t:', acc.result().numpy())
print('Precision\t:', precision)
print('Recall\t\t:', recall)
print('Specificity\t:', spe.result().numpy())
print('F1\t\t:', f1)

## Exporting Model

In [ ]:
model.save(os.path.join('models', 'ODIR_NoDataAug.h5'))

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(os.path.join('models', 'ODIR_NoDataAug.h5'))
model.summary()